# 日よけ効果係数算定ツール

- 一次目標：日よけ効果係数算定ツールのpython上での再現


## 6.3 直達日射に対する日よけ効果係数

### 6.3.1 x+側に対する係数の算定

#### 6.3.1 a) 式(15)

$$
A_{oh0+}(x,y) = \left\{
\begin{array}{ll}
0 \hspace{24pt}(z_{y+}=0)
\\
\frac{1}{2} (x_{3y+} + x_2 / 2 - x) \frac{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{y+} \tan | A_{ZW,j,d,t} |} (x_{3y+} + x_2 / 2 - x)
\\
\hspace{30pt} \left( \begin{array}{ll}
x_{3y+} + x_2 / 2 - x < z_{y+} \tan | A_{ZW,j,d,t} | \\
y_{1} + y_2 / 2 - y \geq \frac{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{y+} \tan | A_{ZW,j,d,t} |} (x_{3y+} + x_2 / 2 - x) 
\end{array} \right)
\\
\Bigl\{ (x_{3y+} + x_2 / 2 - x) - \frac{1}{2} (y_{1} + y_2 / 2 - y) \frac{z_{y+} \tan | A_{ZW,j,d,t} |}{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} \Bigr\} (y_{1} + y_2 / 2 - y)
\\
\hspace{30pt} \left( \begin{array}{ll}
x_{3y+} + x_2 / 2 - x > \frac{z_{y+} \tan | A_{ZW,j,d,t} |}{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} (y_{1} + y_2 / 2 - y) \\
y_{1} + y_2 / 2 - y < z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}
\end{array} \right)
\\
( x_{3y+} + x_2 / 2 - x - \frac{1}{2} z_{y+} \tan | A_{ZW,j,d,t} | ) z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}
\\
\hspace{30pt} \left( \begin{array}{ll}
x_{3y+} + x_2 / 2 - x \geq z_{y+} \tan | A_{ZW,j,d,t} | \\
y_{1} + y_2 / 2 - y \geq z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}
\end{array} \right)
\end{array}
\right.  \qquad (15) 
$$

In [16]:
""" 式(15) """
import numpy as np

def calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs):
 
    X_th = X3yp + X2 / 2 - XX
    Y_th = Y1 + Y2 / 2 - YY
    X_th_Z = Zyp * np.tan(abs(np.radians(Azw)))  
    Y_th_Z = Zyp * np.tan(np.radians(hs)) / np.cos(np.radians(Azw))
        
    if X_th_Z == 0 or Y_th_Z <= 0:
        Aoh0p = 0    # 式(15)条件1 と 日よけが影を落とさない条件をあわせて処理
    else:
        Aoh0p = calc_Aoh0p00(X_th, Y_th, X_th_Z, Y_th_Z)
    return Aoh0p
        
def calc_Aoh0p00(X_th, Y_th, X_th_Z, Y_th_Z):
    
    if (X_th >= X_th_Z and Y_th >= Y_th_Z):
        Aoh0p00 = (X_th - X_th_Z / 2) * Y_th_Z       # 式(15)条件4
    elif Y_th * X_th_Z >= X_th * Y_th_Z:
        Aoh0p00 = X_th ** 2 * Y_th_Z / X_th_Z / 2       # 式(15)条件2
    else:
        Aoh0p00 = (X_th - Y_th / 2 * X_th_Z / Y_th_Z) * Y_th      # 式(15)条件3    
    return Aoh0p00


In [56]:
""" 式(15) Test """
# \確認.xlsx "式(15)Aoh0p"シート

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[1, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0, 0.6, -89, 10, 0]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[2, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -89, 1, 0.0775157853238327]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[3, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -85, 1, 0.0778000319041388]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[4, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -45, 1, 0.0367253907583509]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[5, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -30, 1, 0.0312746037537625]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[6, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -1, 1, 0.0285671195851676]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[7, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -89, 10, 0.783046321489259]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[8, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -85, 10, 0.78591771391823]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[9, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -45, 10, 0.37099130220307]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[10, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -30, 10, 0.315928727588966]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[11, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -1, 10, 0.28857835617943]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[12, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -89, 30, 2.56394116591407]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[13, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -85, 30, 2.57334301233119]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[14, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -45, 30, 1.21474278827522]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[15, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -30, 30, 1.03445051452338]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[16, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -1, 30, 0.944896753480764]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[17, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -89, 60, 6.37950610995132]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[18, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -85, 60, 6.38918763909707]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[19, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -45, 60, 3.64422836482567]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[20, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -30, 60, 3.10335154357014]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[21, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -1, 60, 2.83469026044229]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[22, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -89, 85, 8.62784883202137]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[23, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -85, 85, 8.62931592070401]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[24, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -45, 85, 8.7454171941721]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[25, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -30, 85, 8.82859383225871]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[26, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -1, 85, 9.02239089829093]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[27, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -89, 89, 8.94928560086997]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[28, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -85, 89, 8.9495783031148]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[29, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -45, 89, 8.97274193580687]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[30, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -30, 89, 8.98933669860013]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0pA] = \
[31, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -1, 89, 9.0280015979615]
Aoh0p = calc_Aoh0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aohop = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0p, Aoh0pA, Aoh0p - Aoh0pA ))

case1: Aohop = 0, 期待値 = 0, 残差 = 0
case2: Aohop = 0.0775157853238327, 期待値 = 0.0775157853238327, 残差 = 0.0
case3: Aohop = 0.07780003190413882, 期待値 = 0.0778000319041388, 残差 = 2.7755575615628914e-17
case4: Aohop = 0.03672539075835095, 期待値 = 0.0367253907583509, 残差 = 4.85722573273506e-17
case5: Aohop = 0.03127460375376254, 期待値 = 0.0312746037537625, 残差 = 3.469446951953614e-17
case6: Aohop = 0.028567119585167593, 期待値 = 0.0285671195851676, 残差 = -6.938893903907228e-18
case7: Aohop = 0.7830463214892595, 期待値 = 0.783046321489259, 残差 = 4.440892098500626e-16
case8: Aohop = 0.7859177139182305, 期待値 = 0.78591771391823, 残差 = 5.551115123125783e-16
case9: Aohop = 0.3709913022030704, 期待値 = 0.37099130220307, 残差 = 4.440892098500626e-16
case10: Aohop = 0.3159287275889663, 期待値 = 0.315928727588966, 残差 = 2.7755575615628914e-16
case11: Aohop = 0.28857835617943045, 期待値 = 0.28857835617943, 残差 = 4.440892098500626e-16
case12: Aohop = 2.563941165914074, 期待値 = 2.56394116591407, 残差 = 3.9968028886505635e-15
case13: Aohop =

#### 6.3.1 b) 式(16)

$$
A_{sf0+}(x,y) = \left\{
\begin{array}{ll}
0 \hspace{24pt}(z_{x+}=0)
\\
\frac{1}{2} (y_{1x+} + y_2 / 2 - y) \frac{z_{x+} \tan | A_{ZW,j,d,t} |}{z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} (y_{1x+} + y_2 / 2 - y)
\\
\hspace{30pt} \left( \begin{array}{ll}
y_{1x+} + y_2 / 2 - y < z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t} \\
x_{3} + x_2 / 2 - x \geq \frac{z_{x+} \tan | A_{ZW,j,d,t} |}{z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} (y_{1x+} + y_2 / 2 - y) 
\end{array} \right)
\\
\Bigl\{ (y_{1x+} + y_2 / 2 - y) - \frac{1}{2} (x_{3} + x_2 / 2 - x) \frac{z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{x+} \tan | A_{ZW,j,d,t} |} \Bigr\} (x_{3} + x_2 / 2 - x)
\\
\hspace{30pt} \left( \begin{array}{ll}
y_{1x+} + y_2 / 2 - y > \frac{z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{x+} \tan | A_{ZW,j,d,t} |} (x_{3} + x_2 / 2 - x) \\
x_{3} + x_2 / 2 - x < z_{x+} \tan | A_{ZW,j,d,t} |
\end{array} \right)
\\
( y_{1x+} + y_2 / 2 - y - \frac{1}{2} z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t} ) \; z_{x+} \tan | A_{ZW,j,d,t} |
\\
\hspace{30pt} \left( \begin{array}{ll}
y_{1x+} + y_2 / 2 - y \geq z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t} \\
x_{3} + x_2 / 2 - x \geq z_{x+} \tan | A_{ZW,j,d,t} |
\end{array} \right)
\end{array}
\right.  \qquad (16) 
$$

- コード中では、座標を入れ替えて、`calc_Aoh0p00`を叩くことで対応
  - 式$(15)$の変数 → 式$(16)$の変数
  - $x$ → $y$
  - $x_2$ → $y_2$
  - $x_{3y+}$ → $y_{1x+}$
  - $y$ → $x$
  - $y_1$ → $x_3$
  - $y_2$ → $x_2$
  - $z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}$ → $z_{x+} \tan | A_{ZW,j,d,t} |$
  - $z_{y+} \tan | A_{ZW,j,d,t} |$ → $z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}$

In [54]:
""" 式(16) """
import numpy as np

def calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs):
 
    X_th = Y1xp + Y2 / 2 - YY
    Y_th = X3 + X2 / 2 - XX
    X_th_Z = Zxp * np.tan(np.radians(hs)) / np.cos(np.radians(Azw))  
    Y_th_Z = Zxp * np.tan(abs(np.radians(Azw)))
       
    if X_th_Z == 0 or Y_th_Z <= 0:
        Aoh0p = 0    # 式(16)条件1 と 日よけが影を落とさない条件をあわせて処理
    else:
        Aoh0p = calc_Aoh0p00(X_th, Y_th, X_th_Z, Y_th_Z)
    return Aoh0p

In [71]:
""" 式(16) Test """
# \確認.xlsx "式(16)Asf0p"シート

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[1, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0, 0.52, 0.55, 0.6, -89, 10, 0]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[2, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -89, 1, 8.95144024279148]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[3, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -85, 1, 8.95115216801752]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[4, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -45, 1, 1.44195626616343]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[5, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -30, 1, 0.832815119938684]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[6, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -1, 1, 0.0251839734073206]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[7, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -89, 10, 8.23640771886364]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[8, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -85, 10, 8.23349765491824]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[9, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -45, 10, 1.41607326633252]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[10, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -30, 10, 0.820613756806761]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[11, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -1, 10, 0.0248644613655683]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[12, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -89, 30, 6.43152802877949]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[13, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -85, 30, 6.42199955959408]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[14, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -45, 30, 1.35073959387713]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[15, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -30, 30, 0.789815168251408]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[16, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -1, 30, 0.0240579514858105]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[17, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -89, 60, 2.61502724495748]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[18, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -85, 60, 2.60547310296558]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[19, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -45, 60, 1.16261878163138]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[20, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -30, 60, 0.701134166903881]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[21, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -1, 60, 0.0217356988408539]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[22, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -89, 85, 0.396267657528477]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[23, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -85, 85, 0.394819872434029]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[24, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -45, 85, 0.280246230661627]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[25, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -30, 85, 0.198164010102806]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[26, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -1, 85, 0.00691687769111225]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[27, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -89, 89, 0.0790602737812166]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[28, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -85, 89, 0.0787714228397662]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[29, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -45, 89, 0.0559125714686165]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[30, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -30, 89, 0.0395361584390362]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0pA] = \
[31, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, -1, 89, 0.00138000221209379]
Asf0p = calc_Asf0p(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfop = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0p, Asf0pA, Asf0p - Asf0pA ))


case1: Asfop = 0, 期待値 = 0, 残差 = 0
case2: Asfop = 8.951440242791485, 期待値 = 8.95144024279148, 残差 = 5.329070518200751e-15
case3: Asfop = 8.951152168017515, 期待値 = 8.95115216801752, 残差 = -5.329070518200751e-15
case4: Asfop = 1.4419562661634264, 期待値 = 1.44195626616343, 残差 = -3.552713678800501e-15
case5: Asfop = 0.832815119938684, 期待値 = 0.832815119938684, 残差 = 0.0
case6: Asfop = 0.025183973407320585, 期待値 = 0.0251839734073206, 残差 = -1.3877787807814457e-17
case7: Asfop = 8.236407718863639, 期待値 = 8.23640771886364, 残差 = -1.7763568394002505e-15
case8: Asfop = 8.233497654918239, 期待値 = 8.23349765491824, 残差 = -1.7763568394002505e-15
case9: Asfop = 1.4160732663325195, 期待値 = 1.41607326633252, 残差 = -4.440892098500626e-16
case10: Asfop = 0.820613756806761, 期待値 = 0.820613756806761, 残差 = 0.0
case11: Asfop = 0.02486446136556832, 期待値 = 0.0248644613655683, 残差 = 2.0816681711721685e-17
case12: Asfop = 6.431528028779486, 期待値 = 6.43152802877949, 残差 = -4.440892098500626e-15
case13: Asfop = 6.421999559594079, 期待値 =

#### 6.3.1 c) 式(14)

$$ \begin{align}
A_{wind,j,x+,d,t} &= (x_2 + x_3)(y_1 + y_2) - A_{oh0+}(-x_2 / 2, -y_2 / 2) - A_{sf0+}(-x_2 / 2, -y_2 / 2) \\
&- \{ (x_2 + x_3) y_1 - A_{oh0+}(-x_2 / 2, y_2 / 2) - A_{sf0+}(-x_2 / 2, y_2 / 2) \} \\
&- \{ x_3 (y_1 + y_2) - A_{oh0+}( x_2 / 2, -y_2 / 2) - A_{sf0+}( x_2 / 2, -y_2 / 2) \} \\
&+ x_3 y_1 - A_{oh0+}( x_2 / 2, y_2 / 2) - A_{sf0+}( x_2 / 2, y_2 / 2) \qquad \qquad \qquad (14) \\
\end{align} $$

In [64]:
""" 式(14) """

def calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs):
    Axp = ( (X2 + X3) * (Y1 + Y2) 
           - calc_Aoh0p(-X2/2, -Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) 
           - calc_Asf0p(-X2/2, -Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) )\
        - ( (X2 + X3) * Y1        
           - calc_Aoh0p(-X2/2,  Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) 
           - calc_Asf0p(-X2/2,  Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) ) \
        - ( X3 * (Y1 + Y2)        
           - calc_Aoh0p( X2/2, -Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) 
           - calc_Asf0p( X2/2, -Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) ) \
        + ( X3 * Y1               
           - calc_Aoh0p( X2/2,  Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) 
           - calc_Asf0p( X2/2,  Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) )       
    Axp = max(0, min(Axp, X2 * Y2))    #負値は0に、X2*Y2を超える場合はX2*Y2で頭打ち
    return Axp

In [65]:
""" 式(14) Test """
# \確認.xlsx "式(14)Axp"シート

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[1, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0, 0.28, 0, 0.2, -89, 10, 4.305]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[2, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -89, 1, 6.66133814775094E-16]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[3, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -85, 1, 0.526414267041419]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[4, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -45, 1, 4.305]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[5, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -30, 1, 4.305]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[6, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -1, 1, 4.305]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[7, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -89, 10, 1.88737914186277E-15]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[8, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -85, 10, 0.526414267041418]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[9, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -45, 10, 4.305]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[10, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -30, 10, 4.305]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[11, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -1, 10, 4.305]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[12, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -89, 30, 0.0413341373543166]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[13, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -85, 30, 0.484530517477109]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[14, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -45, 30, 4.305]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[15, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -30, 30, 4.305]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[16, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -1, 30, 4.305]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[17, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -89, 60, 0.0466314443660587]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[18, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -85, 60, 0.0463909860445386]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[19, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -45, 60, 4.305]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[20, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -30, 60, 4.305]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[21, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -1, 60, 4.305]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[22, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -89, 85, 0]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[23, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -85, 85, 9.99200722162641E-16]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[24, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -45, 85, 9.95731275210687E-16]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[25, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -30, 85, 0.542555191463623]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[26, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -1, 85, 1.32157910166076]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[27, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -89, 89, 0]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[28, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -85, 89, 0]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[29, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -45, 89, 0]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[30, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -30, 89, 0]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxpA] = \
[31, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.24, 0.28, 0.21, 0.2, -1, 89, 6.6266436782314E-16]
Axp = calc_Axp(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axp = {}, 期待値 = {}, 残差 = {}'.format( case, Axp, AxpA, Axp - AxpA ))

case1: Axp = 4.305, 期待値 = 4.305, 残差 = 0.0
case2: Axp = 6.661338147750939e-16, 期待値 = 6.66133814775094e-16, 残差 = -9.860761315262648e-32
case3: Axp = 0.5264142670414172, 期待値 = 0.526414267041419, 残差 = -1.7763568394002505e-15
case4: Axp = 4.305, 期待値 = 4.305, 残差 = 0.0
case5: Axp = 4.305, 期待値 = 4.305, 残差 = 0.0
case6: Axp = 4.305, 期待値 = 4.305, 残差 = 0.0
case7: Axp = 1.887379141862766e-15, 期待値 = 1.88737914186277e-15, 残差 = -3.944304526105059e-30
case8: Axp = 0.5264142670414177, 期待値 = 0.526414267041418, 残差 = -3.3306690738754696e-16
case9: Axp = 4.305, 期待値 = 4.305, 残差 = 0.0
case10: Axp = 4.304999999999999, 期待値 = 4.305, 残差 = -8.881784197001252e-16
case11: Axp = 4.305, 期待値 = 4.305, 残差 = 0.0
case12: Axp = 0.04133413735431657, 期待値 = 0.0413341373543166, 残差 = -2.7755575615628914e-17
case13: Axp = 0.48453051747710896, 期待値 = 0.484530517477109, 残差 = -5.551115123125783e-17
case14: Axp = 4.304999999999998, 期待値 = 4.305, 残差 = -1.7763568394002505e-15
case15: Axp = 4.304999999999999, 期待値 = 4.305, 残差 = -8.88178419

### 6.3.2 x-側に対する係数の算定

#### 6.3.2 a) 式(19)

$$
A_{oh0-}(x,y) = \left\{
\begin{array}{ll}
0 \hspace{24pt}(z_{y+}=0)
\\
\frac{1}{2} (x_{1y+} + x_2 / 2 + x) \frac{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{y+} \tan A_{ZW,j,d,t}} (x_{1y+} + x_2 / 2 + x)
\\
\hspace{30pt} \left( \begin{array}{ll}
x_{1y+} + x_2 / 2 + x < z_{y+} \tan A_{ZW,j,d,t} \\
y_{1} + y_2 / 2 - y \geq \frac{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{y+} \tan A_{ZW,j,d,t}} (x_{1y+} + x_2 / 2 + x) 
\end{array} \right)
\\
\Bigl\{ (x_{1y+} + x_2 / 2 + x) - \frac{1}{2} (y_{1} + y_2 / 2 - y) \frac{z_{y+} \tan A_{ZW,j,d,t}}{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} \Bigr\} (y_{1} + y_2 / 2 - y)
\\
\hspace{30pt} \left( \begin{array}{ll}
x_{1y+} + x_2 / 2 + x > \frac{z_{y+} \tan A_{ZW,j,d,t}}{z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} (y_{1} + y_2 / 2 - y) \\
y_{1} + y_2 / 2 - y < z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}
\end{array} \right)
\\
( x_{1y+} + x_2 / 2 + x - \frac{1}{2} z_{y+} \tan A_{ZW,j,d,t} ) z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}
\\
\hspace{30pt} \left( \begin{array}{ll}
x_{1y+} + x_2 / 2 + x \geq z_{y+} \tan A_{ZW,j,d,t} \\
y_{1} + y_2 / 2 - y \geq z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}
\end{array} \right)
\end{array}
\right.  \qquad (19) 
$$

- コード中では、座標を入れ替えて、`calc_Aoh0p00`を叩くことで対応
  - 式$(15)$の変数 → 式$(19)$の変数
  - $x$ → $-x$
  - $x_{3y+}$ → $x_{1y+}$
  - $A_{ZW,j,d,t}$ → $-A_{ZW,j,d,t}$

In [55]:
""" 式(19) """
import numpy as np

def calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs):
 
    X_th = X1yp + X2 / 2 + XX
    Y_th = Y1 + Y2 / 2 - YY
    X_th_Z = Zyp * np.tan(abs(np.radians(Azw)))  
    Y_th_Z = Zyp * np.tan(np.radians(hs)) / np.cos(np.radians(Azw))
        
    if X_th_Z == 0 or Y_th_Z <= 0:
        Aoh0m = 0    # 式(19)条件1 と 日よけが影を落とさない条件をあわせて処理
    else:
        Aoh0m = calc_Aoh0p00(X_th, Y_th, X_th_Z, Y_th_Z)
    return Aoh0m

In [58]:
""" 式(19) Test """
# \確認.xlsx "式(19)Aoh0m"シート

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[1, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0, 0.6, 89, 10, 0]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[2, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 89, 1, 0.00962357025804305]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[3, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 85, 1, 0.00965885941785427]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[4, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 45, 1, 0.010522062047217]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[5, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 30, 1, 0.00987967545219635]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[6, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 1, 1, 0.0100357457458702]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[7, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 89, 10, 0.0972150544392041]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[8, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 85, 10, 0.0975715372725158]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[9, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 45, 10, 0.106291408209752]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[10, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 30, 10, 0.0998021691714904]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[11, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 1, 10, 0.101378754751376]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[12, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 89, 30, 0.318312816474513]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[13, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 85, 30, 0.319480053949725]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[14, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 45, 30, 0.348031667620443]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[15, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 30, 30, 0.326783847856713]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[16, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 1, 30, 0.331946087380636]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[17, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 89, 60, 0.954938449423539]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[18, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 85, 60, 0.958440161849176]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[19, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 45, 60, 1.04409500286133]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[20, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 30, 60, 0.980351543570138]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[21, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 1, 60, 0.995838262141907]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[22, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 89, 85, 2.77994883202137]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[23, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 85, 85, 2.78141592070401]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[24, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 45, 85, 2.8975171941721]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[25, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 30, 85, 2.98069383225871]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[26, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 1, 85, 3.17449089829093]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[27, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 89, 89, 3.10138560086997]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[28, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 85, 89, 3.1016783031148]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[29, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 45, 89, 3.12484193580687]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[30, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 30, 89, 3.14143669860013]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Aoh0mA] = \
[31, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 1, 89, 3.1801015979615]
Aoh0m = calc_Aoh0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Aoh0m = {}, 期待値 = {}, 残差 = {}'.format( case, Aoh0m, Aoh0mA, Aoh0m - Aoh0mA ))

case1: Aoh0m = 0, 期待値 = 0, 残差 = 0
case2: Aoh0m = 0.009623570258043054, 期待値 = 0.00962357025804305, 残差 = 3.469446951953614e-18
case3: Aoh0m = 0.009658859417854266, 期待値 = 0.00965885941785427, 残差 = -5.204170427930421e-18
case4: Aoh0m = 0.010522062047217, 期待値 = 0.010522062047217, 残差 = 0.0
case5: Aoh0m = 0.009879675452196347, 期待値 = 0.00987967545219635, 残差 = -3.469446951953614e-18
case6: Aoh0m = 0.010035745745870175, 期待値 = 0.0100357457458702, 残差 = -2.42861286636753e-17
case7: Aoh0m = 0.09721505443920411, 期待値 = 0.0972150544392041, 残差 = 1.3877787807814457e-17
case8: Aoh0m = 0.09757153727251575, 期待値 = 0.0975715372725158, 残差 = -5.551115123125783e-17
case9: Aoh0m = 0.10629140820975215, 期待値 = 0.106291408209752, 残差 = 1.5265566588595902e-16
case10: Aoh0m = 0.09980216917149039, 期待値 = 0.0998021691714904, 残差 = -1.3877787807814457e-17
case11: Aoh0m = 0.1013787547513757, 期待値 = 0.101378754751376, 残差 = -3.0531133177191805e-16
case12: Aoh0m = 0.31831281647451304, 期待値 = 0.318312816474513, 残差 = 5.5511151231257

#### 6.3.2 b) 式(20)

$$
A_{sf0-}(x, y) = \left\{
\begin{array}{ll}
0 \hspace{24pt}(z_{x-}=0)
\\
\frac{1}{2} (y_{1x-} + y_2 / 2 - y) \frac{z_{x-} \tan A_{ZW,j,d,t}}{z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} (y_{1x-} + y_2 / 2 - y)
\\
\hspace{30pt} \left( \begin{array}{ll}
y_{1x-} + y_2 / 2 - y < z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t} \\
x_{1} + x_2 / 2 + x \geq \frac{z_{x-} \tan A_{ZW,j,d,t}}{z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}} (y_{1x-} + y_2 / 2 - y) 
\end{array} \right)
\\
\Bigl\{ (y_{1x-} + y_2 / 2 - y) - \frac{1}{2} (x_{1} + x_2 / 2 + x) \frac{z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{x-} \tan A_{ZW,j,d,t} } \Bigr\} (x_{1} + x_2 / 2 + x)
\\
\hspace{30pt} \left( \begin{array}{ll}
y_{1x-} + y_2 / 2 - y > \frac{z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}}{z_{x-} \tan A_{ZW,j,d,t}} (x_{1} + x_2 / 2 + x) \\
x_{1} + x_2 / 2 + x < z_{x-} \tan A_{ZW,j,d,t}
\end{array} \right)
\\
( y_{1x-} + y_2 / 2 - y - \frac{1}{2} z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t} ) \; z_{x-} \tan A_{ZW,j,d,t}
\\
\hspace{30pt} \left( \begin{array}{ll}
y_{1x-} + y_2 / 2 - y \geq z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t} \\
x_{1} + x_2 / 2 + x \geq z_{x-} \tan A_{ZW,j,d,t}
\end{array} \right)
\end{array}
\right.  \qquad (20) 
$$

- コード中では、座標を入れ替えて、`calc_Aoh0p00`を叩くことで対応
  - 式$(15)$の変数 → 式$(16)$の変数 → 式$(20)$の変数
  - $x$            → $y$            → $y$
  - $x_2$          → $y_2$          → $y_2$
  - $x_{3y+}$      → $y_{1x+}$      → $y_{1x-}$
  - $y$            → $x$            → $-x$
  - $y_1$          → $x_3$          → $x_1$
  - $y_2$          → $x_2$          → $x_2$
  - $z_{y+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}$ → $z_{x+} \tan | A_{ZW,j,d,t} |$ → $z_{x-} \tan A_{ZW,j,d,t}$
  - $z_{y+} \tan | A_{ZW,j,d,t} |$ → $z_{x+} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}$ → $z_{x-} \tan h_{S,d,t} / \cos A_{ZW,j,d,t}$

In [61]:
""" 式(20) """
import numpy as np

def calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs):
 
    X_th = Y1xm + Y2 / 2 - YY
    Y_th = X1 + X2 / 2 + XX
    X_th_Z = Zxm * np.tan(np.radians(hs)) / np.cos(np.radians(Azw))  
    Y_th_Z = Zxm * np.tan(abs(np.radians(Azw)))
       
    if X_th_Z == 0 or Y_th_Z <= 0:
        Asf0m = 0    # 式(20)条件1 と 日よけが影を落とさない条件をあわせて処理
    else:
        Asf0m = calc_Aoh0p00(X_th, Y_th, X_th_Z, Y_th_Z)
    return Asf0m

In [70]:
""" 式(20) Test """
# \確認.xlsx "式(20)Asf0m"シート

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[1, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0, 0.55, 0.6, 89, 10, 0]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[2, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 89, 1, 3.25643807708641]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[3, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 85, 1, 3.25639934703347]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[4, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 45, 1, 1.54106256237235]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[5, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 30, 1, 0.890086472550928]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[6, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 1, 1, 0.0269164033601141]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[7, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 89, 10, 3.16030592664722]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[8, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 85, 10, 3.15991468471679]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[9, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 45, 10, 1.51068598618192]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[10, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 30, 10, 0.875766817208602]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[11, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 1, 10, 0.0265414204777798]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[12, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 89, 30, 2.9176498794248]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[13, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 85, 30, 2.91636882967876]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[14, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 45, 30, 1.43400966225857]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[15, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 30, 30, 0.8396212514735]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[16, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 1, 30, 0.0255948915216751]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[17, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 89, 60, 2.21894963827439]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[18, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 85, 60, 2.21510648903628]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[19, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 45, 60, 1.21322898677571]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[20, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 30, 60, 0.735544242947583]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[21, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 1, 60, 0.022869470014747]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[22, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 89, 85, 0.385805607034463]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[23, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 85, 85, 0.384396045601409]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[24, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 45, 85, 0.272847316921794]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[25, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 30, 85, 0.192932188023956]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[26, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 1, 85, 0.00673426192045695]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[27, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 89, 89, 0.0769729659713174]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[28, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 85, 89, 0.0766917411206603]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[29, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 45, 89, 0.0544363971333119]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[30, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 30, 89, 0.0384923455563288]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))

[case, XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, Asf0mA] = \
[31, -1.05, -1.025, 1.1, 2.1, 0.9, 1.05, 1.07, 0.88, 0.85, 0.98, 2.05, 1.02, 0.96, 0.92, 1.01, 0.97, 0.48, 0.52, 0.55, 0.6, 1, 89, 0.00134356811874683]
Asf0m = calc_Asf0m(XX, YY, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Asfom = {}, 期待値 = {}, 残差 = {}'.format( case, Asf0m, Asf0mA, Asf0m - Asf0mA ))


case1: Asfom = 0, 期待値 = 0, 残差 = 0
case2: Asfom = 3.2564380770864116, 期待値 = 3.25643807708641, 残差 = 1.7763568394002505e-15
case3: Asfom = 3.2563993470334665, 期待値 = 3.25639934703347, 残差 = -3.552713678800501e-15
case4: Asfom = 1.5410625623723546, 期待値 = 1.54106256237235, 残差 = 4.6629367034256575e-15
case5: Asfom = 0.890086472550928, 期待値 = 0.890086472550928, 残差 = 0.0
case6: Asfom = 0.02691640336011408, 期待値 = 0.0269164033601141, 残差 = -2.0816681711721685e-17
case7: Asfom = 3.160305926647223, 期待値 = 3.16030592664722, 残差 = 3.1086244689504383e-15
case8: Asfom = 3.159914684716786, 期待値 = 3.15991468471679, 残差 = -3.9968028886505635e-15
case9: Asfom = 1.510685986181915, 期待値 = 1.51068598618192, 残差 = -4.884981308350689e-15
case10: Asfom = 0.8757668172086017, 期待値 = 0.875766817208602, 残差 = -3.3306690738754696e-16
case11: Asfom = 0.026541420477779822, 期待値 = 0.0265414204777798, 残差 = 2.0816681711721685e-17
case12: Asfom = 2.9176498794247983, 期待値 = 2.9176498794248, 残差 = -1.7763568394002505e-15
case13: Asfom = 2

#### 6.3.2 c) 式(18)

$$ \begin{align}
A_{wind,j,x-,d,t} &= (x_1 + x_2)(y_1 + y_2) - A_{oh0-}( x_2 / 2, -y_2 / 2) - A_{sf0-}( x_2 / 2, -y_2 / 2) \\
&- \{ (x_1 + x_2) y_1 - A_{oh0-}( x_2 / 2, y_2 / 2) - A_{sf0-}( x_2 / 2, y_2 / 2) \} \\
&- \{ x_1 (y_1 + y_2) - A_{oh0-}(-x_2 / 2, -y_2 / 2) - A_{sf0-}(-x_2 / 2, -y_2 / 2) \} \\
&+ x_1 y_1 - A_{oh0-}(-x_2 / 2, y_2 / 2) - A_{sf0-}(-x_2 / 2, y_2 / 2) \qquad \qquad \qquad (18) \\
\end{align} $$

In [72]:
""" 式(18) """

def calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs):
    Axm = ( (X1 + X2) * (Y1 + Y2) 
           - calc_Aoh0m( X2/2, -Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) 
           - calc_Asf0m( X2/2, -Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) )\
        - ( (X1 + X2) * Y1        
           - calc_Aoh0m( X2/2,  Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) 
           - calc_Asf0m( X2/2,  Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) ) \
        - ( X1 * (Y1 + Y2)        
           - calc_Aoh0m(-X2/2, -Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) 
           - calc_Asf0m(-X2/2, -Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) ) \
        + ( X1 * Y1               
           - calc_Aoh0m(-X2/2,  Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) 
           - calc_Asf0m(-X2/2,  Y2/2, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs) )       
    Axm = max(0, min(Axm, X2 * Y2))    #負値は0に、X2*Y2を超える場合はX2*Y2で頭打ち
    return Axm

In [73]:
""" 式(18) Test """
# \確認.xlsx "式(18)Axm"シート

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[1, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0, 0, 0.2, 89, 10, 4.305]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[2, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 89, 1, 6.66133814775094E-16]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[3, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 85, 1, 0.526414267041419]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[4, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 45, 1, 4.305]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[5, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 30, 1, 4.305]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[6, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 1, 1, 4.305]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[7, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 89, 10, 1.88737914186277E-15]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[8, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 85, 10, 0.526414267041418]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[9, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 45, 10, 4.305]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[10, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 30, 10, 4.305]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[11, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 1, 10, 4.305]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[12, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 89, 30, 0.0413341373543166]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[13, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 85, 30, 0.484530517477109]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[14, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 45, 30, 4.305]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[15, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 30, 30, 4.305]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[16, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 1, 30, 4.305]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[17, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 89, 60, 0.0466314443660587]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[18, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 85, 60, 0.0463909860445386]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[19, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 45, 60, 4.305]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[20, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 30, 60, 4.305]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[21, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 1, 60, 4.305]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[22, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 89, 85, 0]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[23, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 85, 85, 9.99200722162641E-16]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[24, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 45, 85, 9.95731275210687E-16]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[25, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 30, 85, 0.542555191463623]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[26, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 1, 85, 1.32157910166076]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[27, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 89, 89, 0]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[28, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 85, 89, 0]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[29, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 45, 89, 0]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[30, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 30, 89, 0]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

[case, X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs, AxmA] = \
[31, 0.9, 2.1, 1.1, 0.88, 0.85, 1.05, 1.07, 0.98, 2.05, 1.02, 0.92, 0.96, 0.97, 1.01, 0.28, 0.24, 0.21, 0.2, 1, 89, 6.6266436782314E-16]
Axm = calc_Axm(X1, X2, X3, X1yp, X1ym, X3yp, X3ym, Y1, Y2, Y3, Y1xp, Y1xm, Y3xp, Y3xm, Zxp, Zxm, Zyp, Zym, Azw, hs)
print('case{}: Axm = {}, 期待値 = {}, 残差 = {}'.format( case, Axm, AxmA, Axm - AxmA ))

case1: Axm = 4.305, 期待値 = 4.305, 残差 = 0.0
case2: Axm = 6.661338147750939e-16, 期待値 = 6.66133814775094e-16, 残差 = -9.860761315262648e-32
case3: Axm = 0.5264142670414172, 期待値 = 0.526414267041419, 残差 = -1.7763568394002505e-15
case4: Axm = 4.305, 期待値 = 4.305, 残差 = 0.0
case5: Axm = 4.305, 期待値 = 4.305, 残差 = 0.0
case6: Axm = 4.305, 期待値 = 4.305, 残差 = 0.0
case7: Axm = 1.887379141862766e-15, 期待値 = 1.88737914186277e-15, 残差 = -3.944304526105059e-30
case8: Axm = 0.5264142670414177, 期待値 = 0.526414267041418, 残差 = -3.3306690738754696e-16
case9: Axm = 4.305, 期待値 = 4.305, 残差 = 0.0
case10: Axm = 4.304999999999999, 期待値 = 4.305, 残差 = -8.881784197001252e-16
case11: Axm = 4.305, 期待値 = 4.305, 残差 = 0.0
case12: Axm = 0.04133413735431657, 期待値 = 0.0413341373543166, 残差 = -2.7755575615628914e-17
case13: Axm = 0.48453051747710896, 期待値 = 0.484530517477109, 残差 = -5.551115123125783e-17
case14: Axm = 4.304999999999998, 期待値 = 4.305, 残差 = -1.7763568394002505e-15
case15: Axm = 4.304999999999999, 期待値 = 4.305, 残差 = -8.88178419